# Turn raw EEG data into processed (epochs,eyes open, eyes closed, and noise) segments
#### Input: *.edf with recording data and the .csv containing timestamps for resting recording
####  Output: *-preprocessed-epo.fif, *-resting_EC-raw.fif, *resting_EO-raw.fif, *-noise_EO-raw.fif

In [1]:
import os
import mne
import pandas as pd
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
# sys.path.append('/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Code/eeg_toolkit/')
from eeg_toolkit import utils, preprocess, source_localization

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


In [2]:
# Settings
times_tup,time_win_path = preprocess.get_time_window(5)

# Read in data
data_dir = Path('../../Data/')
data_path = data_dir / 'EEG DATA/'
processed_data_path = data_dir / 'Processed Data/'
csv_path = data_dir / 'Eyes Timestamps/'
epo_path = processed_data_path 
save_path_resting = processed_data_path

save_paths=[save_path_resting,]
[os.makedirs(path, exist_ok=True) for path in save_paths];

[-2.5,0.0,2.5]


In [8]:
# All CP and HC subs
sub_ids_CP = [  
    '018',  
    '020',  
    '021',  
    '027',  
    '031',  
    '032',  
    '033',  
    '035',  
    '036',  
    '037',  
    '038',  
    '040',  
    '043',  
    '044',  
    '048',  
    '049',  
    '050',  
    '057',  
    '059',
]

sub_ids_HC = [
    'C10',  
    'C11',  
    'C12',  
    'C13',  
    'C14',  
    'C15',  
    'C16',  
    'C17',  
    'C18',  
    'C19',  
    'C2.',  
    'C24',  
    'C25',  
    'C26',  
    'C27',  
    'C3.',  
    'C6.',  
    'C7.',  
    'C9.'   
]

# CP Subs with widespread pain
sub_ids_WSP = [
    '018',
    '022',
    '024',
    '031',
    '032',
    '034',
    '036',
    '039',
    '040',
    '045',
    '046',
    '052',
]

# CP Subs with localized pain
sub_ids_LP = [  
    '020',  
    '021',  
    '023',  
    '029',  
    '031',  
    '037',  
    '041',  
    '042',  
    '048',  
    '049',  
    '050',  
    '056'  
]  

sub_ids = sub_ids_CP + sub_ids_HC + sub_ids_WSP + sub_ids_LP

# keep unique only
sub_ids = list(set(sub_ids))

In [4]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 43
Controls: 19
Total: 62


In [5]:
# Include noise? if so EO is 3 minutes long. If not, EO is 5 minutes long
include_noise = False

if include_noise:
    save_path = processed_data_path
else:
    save_path = processed_data_path / '5min'
    os.makedirs(save_path, exist_ok=True)

### Run just the 5 min resting crop

In [10]:
for sub_id in sub_ids:
    # Load raw
    raw = mne.io.read_raw_fif(epo_path / f'{sub_id}_preprocessed-raw.fif', preload=True)

    # Preprocess continuous to eyes open, noise calibration, and eyes closed

    # Check if file already exists
    if os.path.isfile(save_path / f'{sub_id}_eyes_open-raw.fif'):
        print(f"{sub_id} already processed")
        continue
    
    _, _, _, = preprocess.get_cropped_resting_EEGs(sub_id, 
                                                    raw,
                                                    csv_path, 
                                                    save_path, 
                                                    include_noise=include_noise)

057 already processed
040 already processed
046 already processed
031 already processed
042 already processed
043 already processed
C9. already processed
C11 already processed
C27 already processed
024 already processed
049 already processed
029 already processed
056 already processed
027 already processed
C17 already processed
038 already processed
C25 already processed
C19 already processed
052 already processed
044 already processed
C26 already processed
039 already processed
023 already processed
050 already processed
034 already processed
C7. already processed
035 already processed
C13 already processed
C2. already processed
022 already processed
C18 already processed
C3. already processed
C12 already processed
021 already processed
C14 already processed
C15 already processed
C6. already processed
020 already processed
041 already processed
C24 already processed
C16 already processed
037 already processed
045 already processed
036 already processed
033 already processed
032 alread

In [7]:
for sub_id in sub_ids:
    # Preprocess continuous to eyes open, noise calibration, and eyes closed
    raw, eyes_closed, noise_segment, eyes_open = preprocess.to_raw(data_path,
                                                                   sub_id,
                                                                   save_path=save_path,
                                                                   csv_path=csv_path,
                                                                   include_noise=include_noise)
    # raw = mne.io.read_raw_fif(epo_path / f'{sub_id}_preprocessed-raw.fif', preload=True)
    
    stim_epochs = mne.read_epochs(epo_path / f'{sub_id}_preprocessed-epo.fif')
    # stim_epochs, epo_times, stim_labels, pain_ratings = preprocess.to_epo(raw, sub_id, data_path, save_path=processed_data_path) 
    
    # check epochs for duration in seconds
    dur = stim_epochs.times[-1] - stim_epochs.times[0]
    print(f"{sub_id} duration: {dur}")
    if dur < 5.0:
        print(f"{sub_id} too short")
        break

018
resampling data from 1000.0 Hz to 400 Hz...


KeyboardInterrupt: 